<a href="https://colab.research.google.com/github/koll-ai/control-meme-api/blob/main/controlmeme_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Install dependencies

!pip install transformers accelerate safetensors xformers opencv-python
!pip install --pre -U triton
!pip install flask-cors
!npm install -g localtunnel

!git clone https://github.com/lllyasviel/ControlNet.git
!git clone https://github.com/koll-ai/control-meme-api.git
!mv control-meme-api/*.py ControlNet/

import sys
import subprocess
import pkg_resources

required = {'torch', 'gradio', 'albumentations', 'opencv-contrib-python', 'imageio', 'imageio-ffmpeg', 'pytorch-lightning', 'omegaconf', 'test-tube', 'streamlit', 'einops', 'transformers', 'webdataset', 'kornia', 'open_clip_torch', 'invisible-watermark', 'streamlit-drawable-canvas', 'torchmetrics', 'timm', 'addict', 'yapf', 'prettytable', 'safetensors', 'basicsr'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

In [18]:
import torch
import cv2
import numpy as np
from PIL import Image

from flask import Flask
from flask import request
import base64
from io import BytesIO

import subprocess
import time
from flask_cors import CORS

import requests

# Download model

In [ ]:
model = "control_sd15_canny" #@param ["control_sd15_mlsd", "control_sd15_hed", "control_sd15_seg", "control_sd15_depth", "control_sd15_scribble", "control_sd15_normal", "control_sd15_openpose", "control_sd15_canny"]

!curl -Lo $m https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/$model
!mv $m /content/ControlNet/models/$model

#detectors = ['body_pose_model.pth', 'dpt_hybrid-midas-501f0c75.pt', 'hand_pose_model.pth', 'mlsd_large_512_fp32.pth', 'mlsd_tiny_512_fp32.pth', 'network-bsds500.pth', 'upernet_global_small.pth']
#for d in detectors:
#  !curl -Lo $d   https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/$d
#  !mv $d /content/ControlNet/annotator/ckpts/$d

# Load model

In [ ]:
%cd /content/ControlNet

import controlmeme
controlmeme.load_model('./models/control_sd15_canny.pth')

In [23]:
import cv2 as cv

def create_hint(image, hint_type):
  init_image = image
  
  if hint_type == 'canny':
    controlnet_hint = Image.fromarray(cv.Canny(np.array(init_image), 100,200))
  
  controlnet_hint.save('last_hint.jpeg')

  return controlnet_hint.convert('RGB')

def generate_controlnet(prompt, hint, num_inference_steps, seed=-1, negative_prompt="", **kwargs):
    """
    Generate a meme variation
    POST params: prompt:str, negative_prompt:str, controlnet_hint_url:str, num_inference_steps:int, seed:int
    """
    
    hint = np.asarray(hint)

    output = controlmeme.generate(hint, prompt, "", negative_prompt, 1, 512, num_inference_steps, False, 1, 9, seed, 0)

    return Image.fromarray(output[0])

In [26]:
with open('output.txt', 'w') as f:
    proc = subprocess.Popen(['lt', '--port', '5000'], stdout=f)

time.sleep(3)

app = Flask(__name__)
CORS(app)

params = dict()

API_URL = ""

@app.route('/hello/')
def hello():    
    return "hello"

@app.route('/', methods=['POST'])
def index():
    global params

    params = request.get_json()

    meme_url = params['controlnet_hint_url']
    image = Image.open(requests.get(meme_url, stream=True).raw)
    hint = create_hint(image, 'canny')

    last_image = generate_controlnet(hint=hint, **params)

    with open('./last_meme.jpeg', 'w') as f:
      image.save(f, format="JPEG")

    buffered = BytesIO()
    last_image.save(buffered, format="JPEG")

    return base64.b64encode(buffered.getvalue())


@app.route('/save_last/')
def save_last():
    requests.post(f"{API_URL}/api/meme/{params['uuid']}/variation/",
                  files=dict(
                      file=open('./last.jpeg'),
                      prompt=params["prompt"],
                      nb_steps=params["num_inference_steps"]
                      )
                  )    
    return "ok"

if __name__ == '__main__':
  with open('output.txt', 'r') as f:
      contents = f.read()
      API_URL = contents.split(' ')[-1]
      protocol = API_URL.split(':')[0]
      url = API_URL.split('/')[-1]

      print(API_URL)

      print('\n'*2)
      print(f'Everything is ready! Click on the this link to be redirected to koll.ai. Don\'t close this tab!')
      print(f'https://meme.koll.ai?protocol={protocol}&url={url}')

  app.run()

https://thick-crabs-lie-34-91-160-60.loca.lt




Everything is ready! Click on the this link to be redirected to koll.ai. Don't close this tab!
https://meme.koll.ai?protocol=https&url=thick-crabs-lie-34-91-160-60.loca.lt

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [26/Feb/2023 22:02:24] "OPTIONS / HTTP/1.1" 200 -
INFO:lightning_fabric.utilities.seed:Global seed set to 45000


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0
Running DDIM Sampling with 10 timesteps


DDIM Sampler: 100%|██████████| 10/10 [00:07<00:00,  1.28it/s]
INFO:werkzeug:127.0.0.1 - - [26/Feb/2023 22:02:35] "POST / HTTP/1.1" 200 -


In [ ]:
meme_url = "https://storage.googleapis.com/control-meme-public/hidethepainharold.jpg"

image = Image.open(requests.get(meme_url, stream=True).raw)
image

# get controlnet hint image
hint = create_hint(image, 'canny')
hint

params = {"prompt": "Macron",
          "hint": hint,
          "num_inference_steps": 50
        }

b64 = generate_controlnet(**params)
b64